In [1]:
import numpy as np
import scipy
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn import preprocessing 

%matplotlib inline

In [2]:
df_train = pd.read_csv('training_variants')
df_test = pd.read_csv('test_variants')
combined = [df_train,df_test]

In [10]:
df_train.head(10)

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4
5,5,CBL,V391I,4
6,6,CBL,V430M,5
7,7,CBL,Deletion,1
8,8,CBL,Y371H,4
9,9,CBL,C384R,4


In [45]:
df_train['Variation'].describe()
# 9 classes exist
# 264 genes
# 2996 variations

count                     3321
unique                    2996
top       Truncating Mutations
freq                        93
Name: Variation, dtype: object

In [46]:
df_test['Variation'].describe()
# 1397 genes
# 5628 variations
# test data far bigger with more categories in both genes and variations. Do training data have enough info to
# predict these "unknown" variations?
# how to model "clinical evidences (text)"?

count                     5668
unique                    5628
top       Truncating Mutations
freq                        18
Name: Variation, dtype: object

In [3]:
df_train_txt = pd.read_csv('training_text',sep="|",names = ['Text'],skiprows=[0],usecols=[2],skipinitialspace=True)
df_test_txt = pd.read_csv('test_text',sep="|",names = ['Text'],skiprows=[0],usecols=[2],skipinitialspace=True)

In [38]:
df_train_txt['Text'].describe()
# training has 1831 different clinical evidences

count                                                  3321
unique                                                 1831
top       Mutation screening of the breast and ovarian c...
freq                                                     64
Name: Text, dtype: object

In [40]:
df_test_txt['Text'].describe()
# test has 5586 different clinical evidences

count                                                  5667
unique                                                 5586
top       Mutations in BRCA1 and BRCA2 account for the m...
freq                                                      9
Name: Text, dtype: object

In [72]:
df_train_txt.head(2)

,Text,ID
0,Cyclin-dependent kinases (CDKs) regulate a var...,0
1,Abstract Background Non-small cell lung cance...,1


In [4]:
df_train_txt['ID']=range(len(df_train_txt))
df_test_txt['ID']=range(len(df_test_txt))

In [92]:
df_train_txt['prt'] = df_train_txt['Text'].str.extract(r"\(([A-Za-z0-9_/]+)\)")

/Users/admin/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [95]:
df_train_txt['prt'].describe()

count     3298
unique     696
top          1
freq       158
Name: prt, dtype: object

In [107]:
df_test_txt['prt'] = df_test_txt['Text'].str.extract(r"\(([A-Za-z_/0-9]+)\)")

/Users/admin/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [89]:
df_test_txt['prt'].describe()

count     5643
unique    1240
top          1
freq       215
Name: prt, dtype: object

In [108]:
df_test_txt['prt'].tail(40)

5628        IR
5629       JNK
5630       T58
5631       YAP
5632       PCR
5633        AR
5634      1995
5635      EGFR
5636       GAP
5637      CIMP
5638        CR
5639         i
5640     FGFR4
5641        PH
5642     NSCLC
5643      PRC2
5644      IDH1
5645       FAL
5646         3
5647     HNPCC
5648         1
5649    D1028N
5650      DBSs
5651     PTPRD
5652      EGFR
5653        PI
5654       FSH
5655        kb
5656       ALT
5657        BC
5658       LOH
5659     HNPCC
5660       GBM
5661     DLBCL
5662     SINEs
5663       Q61
5664       MDS
5665      BAP1
5666       NF2
5667       MMR
Name: prt, dtype: object